In [1]:
import pathlib
import astropy.table as at
import ezpadova as ez
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import KDTree
from scipy.interpolate import InterpolatedUnivariateSpline

%matplotlib inline

In [2]:
data_path = pathlib.Path("../data")
data_path.mkdir(exist_ok=True)

In [3]:
teff_grid = np.round(np.arange(4800, 5500 + 1e-3, 100), 0)
logg_grid = np.round(np.arange(2, 3.0 + 1e-3, 0.2), 1)
mh_grid = np.round(np.arange(-0.5, 0.5 + 1e-3, 0.1), 1)

arrs = np.meshgrid(teff_grid, logg_grid, mh_grid, indexing="ij")
arrs[0].size

528

In [4]:
np.savez(data_path / "teff_logg_mh_grids.npz", teff_grid, logg_grid, mh_grid)

In [5]:
filename = data_path / "grid_params.csv"

tbl = at.Table()
tbl["Teff"] = arrs[0].flatten()
tbl["logg"] = arrs[1].flatten()
tbl["m_H"] = arrs[2].flatten()
tbl["a_H"] = 0.2
tbl.write(filename, overwrite=True)

In [6]:
print(f"Now run:\n\t julia scripts/make_korg_grid.jl data/{filename.name}")

Now run:
	 julia scripts/make_korg_grid.jl data/grid_params.csv


# Synthesize the one test case, off the main grid:

In [11]:
rng = np.random.default_rng(42)
teff = np.mean(teff_grid) + rng.uniform(0, np.diff(teff_grid)[0])
logg = np.mean(logg_grid) + rng.uniform(0, np.diff(logg_grid)[0])
mh = np.mean(mh_grid) + rng.uniform(0, np.diff(mh_grid)[0])

true_params = np.array([[teff, logg, mh], [teff_grid[2], logg_grid[2], mh_grid[2]]])
np.save(data_path / "true_params.npy", true_params)

In [9]:
test_filename = data_path / "test_params.csv"

tbl = at.Table()
tbl["Teff"] = np.array([teff, teff_grid[2]])
tbl["logg"] = [logg, logg_grid[2]]
tbl["m_H"] = [mh, mh_grid[2]]
tbl["a_H"] = 0.2
tbl.write(test_filename, overwrite=True)

In [10]:
print(f"Now run:\n\t julia scripts/make_korg_grid.jl data/{test_filename.name}")

Now run:
	 julia scripts/make_korg_grid.jl data/test_params.csv
